In [1]:
import os
currentDirectory = os.getcwd()
data_dir = currentDirectory[:-7] + '/input/'
print(data_dir)

from xmitgcm import open_mdsdataset
ds1 = open_mdsdataset(data_dir, geometry='cartesian', endian='<',prefix=['statevars'],iters=range(100*1860,170*1860,1860))

/scratch/jxchang/HighRes1/results/TideU008N0LinH200ho140Ah0200Cdqdt003/input/


In [2]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import cm
from matplotlib.colors import BoundaryNorm
import matplotlib.animation as animation
import math

In [3]:
t = 0
h0 = 140.

f0 = 1.e-4
g = 9.8
u0=0.08

om=2*np.pi/12.4/3600 
alpha = 2e-4 

tR_fname="../indata/TRef.bin"
tRef = np.fromfile(tR_fname)
refSalt=35.
refTemp=tRef[0]
print('refTemp='+ str(refTemp))
print('tRef='+ str(tRef))

xmin = -4
xmax = 4
vmax = 3
vmin = -3
wmax = 0.2
wmin = -0.2
tmax = math.floor(np.amax(tRef))
tmin = math.floor(np.amin(tRef))
numcolt=21
numcolv=30

ttlen=len(ds1.time)
print('the length of time:' + str(ttlen) )
print(tmin)

refTemp=63.59230581861823
tRef=[63.59230582 40.69584406 33.47293155 31.54712706 29.26896757 28.65696899
 28.23721628 27.65390158 27.08300659 26.5118492  25.90295299 25.69407634
 25.29910279 24.81832859 24.41499174 24.26210069 24.21619203 24.14261024
 24.0996551  24.02495734 23.98958682 23.95165181 23.93411373 23.9224752
 23.91844136 23.91059116 23.87093593 23.86487752 23.86221099 23.85790713
 23.84947122 23.84359894 23.82719386 23.81078878 23.79438371 23.77797863
 23.76157355 23.74516847 23.72876339 23.71235831]
the length of time:70
23


In [4]:
xg=ds1.coords["XG"]
xc=ds1.coords["XC"]
yg=ds1.coords["YG"]
yc=ds1.coords["YC"]
z=ds1.coords["Z"]
time=ds1.coords['time'].values/np.timedelta64(1, 's')/3600
ds1=ds1.assign_coords(XC0 = (ds1.XC/1000.-ds1.XC.mean()/1000.))
ds1=ds1.assign_coords(XG0 = (ds1.XG-ds1.XC.mean())/1000.)
ds1=ds1.assign_coords(YC0 = (ds1.YC/1000.-ds1.YC.mean()/1000.))
ds1=ds1.assign_coords(YG0 = (ds1.YG-ds1.YC.mean())/1000.)
yc0 = (ds1.YC/1000.-ds1.YC.mean()/1000.)
xc0 = (ds1.XC/1000.-ds1.XC.mean()/1000.)

In [5]:
fig = plt.figure(figsize=(11,8))
grid = plt.GridSpec(2, 2, height_ratios=[0.6,2.5], width_ratios=[1,0.1],wspace=0.02,hspace=0.38)

<Figure size 792x576 with 0 Axes>

In [6]:
SMALL_SIZE = 17
MEDIUM_SIZE = 19
BIGGER_SIZE = 21

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

In [7]:
#plot bathymetry
ax1=fig.add_subplot(grid[0,:-1])
ax1.plot(time,np.zeros(ttlen),color='gray',linestyle='--')
ax1.plot(time,ds1.UVEL.isel(Z=0,YC=0,XG=0),label='U(t,0,0,0)')
ax1.plot(time,u0*np.cos(om*ds1.time.values/np.timedelta64(1, 's')),label='Uinput')
ax1.set_xlim(min(time), max(time))
ax1.set_ylim(-u0-0.05, u0+0.05)
point, = ax1.plot([time[0]],[ds1.UVEL.isel(time=t,YC=40,XG=0,Z=0).data], 'go', color='red',ms=10)
ax1.set_xlabel('time [hr]')
ax1.set_ylabel('U [m/s]')
ax1.set_title('Uinput vs U[t,0,0,0] ', loc='left')



Text(0.0, 1.0, 'Uinput vs U[t,0,0,0] ')

In [8]:
# plot temperature
ax2=fig.add_subplot(grid[1,0])
cmap = cm.Spectral_r
levels = np.linspace(23, 50, num=numcolt)
norm = BoundaryNorm(levels, ncolors=cmap.N, clip=True)
opts = {'cmap':cm.get_cmap(cmap, len(levels) - 1), 'norm': norm}

cset1 = ax2.contourf(ds1.XC0, ds1.Z, ds1.THETA.isel(time=t,YC=60), levels , extend='max', **opts)

cax = fig.add_subplot(grid[1,-1])
cax.set_visible(False)
fig.colorbar(cset1, ax=cax,extend='both')
#fig.colorbar(cset1, ax=axs,ticks=np.arange(tmin, tmax+0.1, 5))
ax2.set_xlabel('X [km]')
ax2.set_ylabel('Z [m]')
ax2.set_title('T [oC]', loc='left')
ax2.set(xlim=(xmin, xmax),ylim=(-208,0))
ax2.plot(ds1['XC0'],-ds1.Depth.isel(YC=60),lw=1.5)

/home/jxchang/jupyter_py3.7/lib/python3.7/site-packages/ipykernel_launcher.py:12: MatplotlibDeprecationWarning: The 'extend' parameter to Colorbar has no effect because it is overridden by the mappable; it is deprecated since 3.3 and will be removed two minor releases later.
  if sys.path[0] == '':


In [9]:
from scipy.interpolate import griddata
print(xc.shape)
Xu,Zu=np.meshgrid(xg.values,ds1.Z.values)
Xw,Zw=np.meshgrid(xc.values,ds1.Zl.values)
print(np.shape(Xu))
points_u = np.transpose([Xu.flatten(),Zu.flatten()])
points_w = np.transpose([Xw.flatten(),Zw.flatten()])
print(np.shape(points_u))
grid_x, grid_z = np.mgrid[36000:44000:200,-200:0:5]
grid_U = griddata(points_u,(ds1.UVEL.isel(time=t,YC=60).values).flatten(),(grid_x, grid_z))
grid_w = griddata(points_w,(ds1.WVEL.isel(time=t,YC=60).values).flatten(),(grid_x, grid_z))
grid_x0=grid_x/1000.-ds1.XC.values.mean()/1000.
grid_z0=grid_z
print(grid_x0[0,:5])

(1440,)
(40, 1440)
(57600, 2)
[-4. -4. -4. -4. -4.]


In [10]:
quiv = ax2.quiver(grid_x0, grid_z0, grid_U, grid_w, color='k', units='y',scale=0.08)
ax2.quiverkey(quiv,0.93,0.02,1,r'$1 \frac{m}{s}$', labelpos='E')

In [11]:
fig
print(ax2.collections)
print(np.size(ax2.collections))

[<matplotlib.collections.PathCollection object at 0x2af4b6d50e50>, <matplotlib.collections.PathCollection object at 0x2af4b6d539d0>, <matplotlib.collections.PathCollection object at 0x2af4b6d53a90>, <matplotlib.collections.PathCollection object at 0x2af4b6d53e10>, <matplotlib.collections.PathCollection object at 0x2af4b6d691d0>, <matplotlib.collections.PathCollection object at 0x2af4b6d69550>, <matplotlib.collections.PathCollection object at 0x2af4b6d69950>, <matplotlib.collections.PathCollection object at 0x2af4b6d69cd0>, <matplotlib.collections.PathCollection object at 0x2af4b6d7f090>, <matplotlib.collections.PathCollection object at 0x2af4b6d69590>, <matplotlib.collections.PathCollection object at 0x2af4b6d7f710>, <matplotlib.collections.PathCollection object at 0x2af4b6d7fa90>, <matplotlib.collections.PathCollection object at 0x2af4b6d7fe10>, <matplotlib.collections.PathCollection object at 0x2af4b6d891d0>, <matplotlib.collections.PathCollection object at 0x2af4b6d89590>, <matplotl

In [12]:
tt: int=0
def updateData(tt):

    grid_U = griddata(points_u,(ds1.UVEL.isel(time=tt,YC=60).values).flatten(),(grid_x, grid_z))
    grid_w = griddata(points_w,(ds1.WVEL.isel(time=tt,YC=60).values).flatten(),(grid_x, grid_z))
    updatetime = time[tt]
    print(tt)
    print(np.nanmax(grid_U))
    print(np.nanmax(grid_w))
    ax2.collections[0:numcolt] = []
    ax2.contourf(ds1.XC0, ds1.Z, ds1.THETA.isel(time=tt,YC=60), levels, extend='max', **opts)
    print(np.size(ax2.collections))
    print(ax2.collections)
    ax2.collections.append(ax2.collections.pop(0))
    print(ax2.collections)
    quiv.set_UVC(grid_U,grid_w)
    print(ax2.collections)
    point.set_data(updatetime,ds1.UVEL.isel(time=tt,YC=60,XG=0,Z=0))
    ax1.set_title('t= %2.3fhr' % updatetime)
    fig.savefig('./figsMag2/summary_TUvec_y60_xz_t%s.png' %(tt+100))



simulation = animation.FuncAnimation(fig, updateData, blit=False, frames= 40 , interval=40, repeat=False)
plt.draw()
plt.show()
simulation.save(filename='summary_TUvec_y60_xz.gif',writer='imagemagick', fps=2)

<Figure size 432x288 with 0 Axes>

0
1.035561578723069
0.14402125285544787
22
[<matplotlib.quiver.Quiver object at 0x2af4b6548490>, <matplotlib.collections.PathCollection object at 0x2af4b7f0ea10>, <matplotlib.collections.PathCollection object at 0x2af4b7f05950>, <matplotlib.collections.PathCollection object at 0x2af4b7f0ef90>, <matplotlib.collections.PathCollection object at 0x2af4b7f17350>, <matplotlib.collections.PathCollection object at 0x2af4b7f17710>, <matplotlib.collections.PathCollection object at 0x2af4b7f17a50>, <matplotlib.collections.PathCollection object at 0x2af4b7f17c10>, <matplotlib.collections.PathCollection object at 0x2af4b7f17f90>, <matplotlib.collections.PathCollection object at 0x2af4b7f23310>, <matplotlib.collections.PathCollection object at 0x2af4b7f17a90>, <matplotlib.collections.PathCollection object at 0x2af4b7f239d0>, <matplotlib.collections.PathCollection object at 0x2af4b7f23d50>, <matplotlib.collections.PathCollection object at 0x2af4b7f2f110>, <matplotlib.collections.PathCollection object

1
0.8992295469221534
0.29122899704351013
22
[<matplotlib.quiver.Quiver object at 0x2af4b6548490>, <matplotlib.collections.PathCollection object at 0x2af4bc18b950>, <matplotlib.collections.PathCollection object at 0x2af4bc18bfd0>, <matplotlib.collections.PathCollection object at 0x2af4bc199190>, <matplotlib.collections.PathCollection object at 0x2af4bc199510>, <matplotlib.collections.PathCollection object at 0x2af4bc199890>, <matplotlib.collections.PathCollection object at 0x2af4bc199d50>, <matplotlib.collections.PathCollection object at 0x2af4bc1a0150>, <matplotlib.collections.PathCollection object at 0x2af4bc1a0390>, <matplotlib.collections.PathCollection object at 0x2af4b7eec710>, <matplotlib.collections.PathCollection object at 0x2af4bc1994d0>, <matplotlib.collections.PathCollection object at 0x2af4bc000b10>, <matplotlib.collections.PathCollection object at 0x2af4bc000c90>, <matplotlib.collections.PathCollection object at 0x2af4bc177190>, <matplotlib.collections.PathCollection objec

3
0.4059617856608859
0.09291687129495345
22
[<matplotlib.quiver.Quiver object at 0x2af4b6548490>, <matplotlib.collections.PathCollection object at 0x2af4bc16c390>, <matplotlib.collections.PathCollection object at 0x2af4bc119710>, <matplotlib.collections.PathCollection object at 0x2af4bc18b290>, <matplotlib.collections.PathCollection object at 0x2af4bc1771d0>, <matplotlib.collections.PathCollection object at 0x2af4bc177c50>, <matplotlib.collections.PathCollection object at 0x2af4bc177f90>, <matplotlib.collections.PathCollection object at 0x2af4bc177510>, <matplotlib.collections.PathCollection object at 0x2af4bc200d10>, <matplotlib.collections.PathCollection object at 0x2af4bc200fd0>, <matplotlib.collections.PathCollection object at 0x2af4bc177fd0>, <matplotlib.collections.PathCollection object at 0x2af4bc203190>, <matplotlib.collections.PathCollection object at 0x2af4bc2034d0>, <matplotlib.collections.PathCollection object at 0x2af4bc203890>, <matplotlib.collections.PathCollection objec

5
0.230902852123958
0.03366720613604452
22
[<matplotlib.quiver.Quiver object at 0x2af4b6548490>, <matplotlib.collections.PathCollection object at 0x2af4bc18b190>, <matplotlib.collections.PathCollection object at 0x2af4b7f90f10>, <matplotlib.collections.PathCollection object at 0x2af4bc200ed0>, <matplotlib.collections.PathCollection object at 0x2af4bc176ad0>, <matplotlib.collections.PathCollection object at 0x2af4bc18d390>, <matplotlib.collections.PathCollection object at 0x2af4bc18d690>, <matplotlib.collections.PathCollection object at 0x2af4bc206110>, <matplotlib.collections.PathCollection object at 0x2af4bc1d6290>, <matplotlib.collections.PathCollection object at 0x2af4bc1d6050>, <matplotlib.collections.PathCollection object at 0x2af4b7f43990>, <matplotlib.collections.PathCollection object at 0x2af4bc1a81d0>, <matplotlib.collections.PathCollection object at 0x2af4bc1a0390>, <matplotlib.collections.PathCollection object at 0x2af4b7fbdd90>, <matplotlib.collections.PathCollection object

...

31
0.14437824952990458
0.06951002649517275
22
[<matplotlib.quiver.Quiver object at 0x2af4b6548490>, <matplotlib.collections.PathCollection object at 0x2af4bc20ef50>, <matplotlib.collections.PathCollection object at 0x2af4bc20ee90>, <matplotlib.collections.PathCollection object at 0x2af4bc1b22d0>, <matplotlib.collections.PathCollection object at 0x2af4bc1b2590>, <matplotlib.collections.PathCollection object at 0x2af4bc161890>, <matplotlib.collections.PathCollection object at 0x2af4bc161490>, <matplotlib.collections.PathCollection object at 0x2af4bc161c50>, <matplotlib.collections.PathCollection object at 0x2af4bc018a50>, <matplotlib.collections.PathCollection object at 0x2af4bc1d3b10>, <matplotlib.collections.PathCollection object at 0x2af4bc161a90>, <matplotlib.collections.PathCollection object at 0x2af4bc1d3910>, <matplotlib.collections.PathCollection object at 0x2af4bc1d3190>, <matplotlib.collections.PathCollection object at 0x2af4bc18d410>, <matplotlib.collections.PathCollection obj

33
0.17890105370501208
0.2670439607395717
22
[<matplotlib.quiver.Quiver object at 0x2af4b6548490>, <matplotlib.collections.PathCollection object at 0x2af4bc028b90>, <matplotlib.collections.PathCollection object at 0x2af4bc018890>, <matplotlib.collections.PathCollection object at 0x2af4bc028d10>, <matplotlib.collections.PathCollection object at 0x2af4bc005790>, <matplotlib.collections.PathCollection object at 0x2af4bc005dd0>, <matplotlib.collections.PathCollection object at 0x2af4bc005450>, <matplotlib.collections.PathCollection object at 0x2af4bc005c10>, <matplotlib.collections.PathCollection object at 0x2af4bc2065d0>, <matplotlib.collections.PathCollection object at 0x2af4b7f51390>, <matplotlib.collections.PathCollection object at 0x2af4bc005750>, <matplotlib.collections.PathCollection object at 0x2af4bc2c9990>, <matplotlib.collections.PathCollection object at 0x2af4b7f0ea50>, <matplotlib.collections.PathCollection object at 0x2af4bc20e8d0>, <matplotlib.collections.PathCollection obje

35
0.32557041671397297
0.21143411903961734
22
[<matplotlib.quiver.Quiver object at 0x2af4b6548490>, <matplotlib.collections.PathCollection object at 0x2af4bc2eb4d0>, <matplotlib.collections.PathCollection object at 0x2af4b6e3f350>, <matplotlib.collections.PathCollection object at 0x2af4bc005990>, <matplotlib.collections.PathCollection object at 0x2af4bc005650>, <matplotlib.collections.PathCollection object at 0x2af4bc230ed0>, <matplotlib.collections.PathCollection object at 0x2af4bc230f50>, <matplotlib.collections.PathCollection object at 0x2af4bc230c90>, <matplotlib.collections.PathCollection object at 0x2af4bc230a90>, <matplotlib.collections.PathCollection object at 0x2af4bc2b0990>, <matplotlib.collections.PathCollection object at 0x2af4bc230150>, <matplotlib.collections.PathCollection object at 0x2af4bc2eb090>, <matplotlib.collections.PathCollection object at 0x2af4bc20e390>, <matplotlib.collections.PathCollection object at 0x2af4bc058390>, <matplotlib.collections.PathCollection obj

37
0.23524977360853863
0.2669517186750487
22
[<matplotlib.quiver.Quiver object at 0x2af4b6548490>, <matplotlib.collections.PathCollection object at 0x2af4b7f0a510>, <matplotlib.collections.PathCollection object at 0x2af4bc279250>, <matplotlib.collections.PathCollection object at 0x2af4bc1e1390>, <matplotlib.collections.PathCollection object at 0x2af4bc1e1650>, <matplotlib.collections.PathCollection object at 0x2af4bc1e79d0>, <matplotlib.collections.PathCollection object at 0x2af4bc1e7d10>, <matplotlib.collections.PathCollection object at 0x2af4bc2e7150>, <matplotlib.collections.PathCollection object at 0x2af4bc2e7290>, <matplotlib.collections.PathCollection object at 0x2af4bc2e75d0>, <matplotlib.collections.PathCollection object at 0x2af4bc1e7d50>, <matplotlib.collections.PathCollection object at 0x2af4bc026d90>, <matplotlib.collections.PathCollection object at 0x2af4bc199a90>, <matplotlib.collections.PathCollection object at 0x2af4bc199310>, <matplotlib.collections.PathCollection obje

39
0.3214315320385075
0.10700574653532685
22
[<matplotlib.quiver.Quiver object at 0x2af4b6548490>, <matplotlib.collections.PathCollection object at 0x2af4bc026290>, <matplotlib.collections.PathCollection object at 0x2af4bc058690>, <matplotlib.collections.PathCollection object at 0x2af4bc026750>, <matplotlib.collections.PathCollection object at 0x2af4bc1afe50>, <matplotlib.collections.PathCollection object at 0x2af4bc1af810>, <matplotlib.collections.PathCollection object at 0x2af4bc005810>, <matplotlib.collections.PathCollection object at 0x2af4b6e33fd0>, <matplotlib.collections.PathCollection object at 0x2af4bc1b1150>, <matplotlib.collections.PathCollection object at 0x2af4bc1b1f90>, <matplotlib.collections.PathCollection object at 0x2af4bc005610>, <matplotlib.collections.PathCollection object at 0x2af4bc1b1a90>, <matplotlib.collections.PathCollection object at 0x2af4bc1b1d50>, <matplotlib.collections.PathCollection object at 0x2af4b5110290>, <matplotlib.collections.PathCollection obje

In [13]:
from IPython.display import Image
from IPython.display import display
display(Image(url='./summary_TUvec_y60_xz.gif'))